# Demo – Clasificador de documentos testimoniales

**Autor:** Manuel Daza Ramirez  
**Proyecto:** Demo – UBPD: Clasificador de documentos testimoniales

---

## Contexto institucional

La **UBPD (Unidad de Búsqueda de Personas dadas por Desaparecidas)** es una entidad del Estado colombiano creada en el marco del Acuerdo de Paz de 2016. Su misión es dirigir, coordinar y contribuir a la implementación de acciones humanitarias de búsqueda de personas dadas por desaparecidas en el contexto y en razón del conflicto armado.

La UBPD recibe miles de documentos testimoniales de familiares de víctimas, organizaciones sociales y otras fuentes. Estos documentos contienen información crítica sobre:
- **Hechos victimizantes** (desaparición forzada, masacres, desplazamiento)
- **Actores armados** involucrados (guerrillas, paramilitares, fuerza pública)
- **Territorios** donde ocurrieron los hechos
- **Períodos temporales** del conflicto

---

## Problema que resuelve este prototipo

La clasificación manual de testimonios presenta varios desafíos:

1. **Volumen**: Miles de documentos requieren clasificación.
2. **Inconsistencia**: Diferentes analistas pueden clasificar el mismo documento de formas distintas.
3. **Tiempo**: La clasificación manual consume recursos humanos escasos.
4. **Priorización**: Es difícil identificar rápidamente casos urgentes que requieren atención inmediata.

Este prototipo utiliza un **LLM (Large Language Model)** con una **ontología controlada** para:
- Clasificar automáticamente documentos según categorías predefinidas
- Garantizar consistencia mediante vocabularios controlados
- Calcular scores de prioridad para enrutamiento
- Extraer fragmentos relevantes (highlights) para análisis posterior

---

## Objetivo técnico

- Cargar la ontología UBPD desde un archivo YAML.
- Construir prompts estructurados (system + user) que fuerzan al LLM a devolver un JSON con etiquetas controladas.
- Implementar funciones de preprocesamiento, llamada al modelo, parsing de JSON y validación de etiquetas.
- Exponer una función principal `classify_document(text)` y un ejemplo de uso con un testimonio de prueba.

## Estructura del notebook

1. **Configuración del entorno** - Cliente OpenAI y variables globales
2. **Preprocesamiento de texto** - Limpieza y normalización de documentos
3. **Carga de ontología** - YAML → estructura Python
4. **Serialización para prompts** - Ontología → texto legible para el LLM
5. **Definición de prompts** - System prompt con reglas + User template con ejemplos
6. **Cliente LLM** - Función de llamada al modelo
7. **Extracción de JSON** - Parsing seguro de respuestas
8. **Validación y normalización** - Corrección de etiquetas inválidas
9. **Cálculo de prioridad** - Score para enrutamiento de casos
10. **Función principal** - Pipeline completo de clasificación
11. **Ejemplo de uso** - Demostración con testimonio sintético

> **Nota**: Este notebook es un demo. En producción se integraría con bases de datos, mecanismos de auditoría, logging estructurado y monitoreo de calidad del modelo.

---
## 1. Configuración inicial del entorno

Esta sección establece las dependencias y configuración necesarias para el funcionamiento del clasificador.

### Decisiones de diseño:
- **`python-dotenv`**: Permite cargar la API key desde un archivo `.env`, evitando hardcodear credenciales en el código.
- **`dataclass`**: Aunque no se usa extensivamente aquí, facilita la creación de estructuras de datos tipadas en Python.
- **Cliente OpenAI**: Se usa el SDK oficial de OpenAI. En producción, podría sustituirse por Azure OpenAI, Anthropic Claude, o modelos locales.

### Configuración del modelo:
- **`temperature=0.0`** (en la llamada): Respuestas determinísticas para clasificación consistente.
- El modelo se especifica globalmente para facilitar cambios (ej. migrar de GPT-4 a GPT-4o).

In [ ]:
# ==============================================================================
# SECCIÓN 1: CONFIGURACIÓN INICIAL DEL ENTORNO
# ==============================================================================
# Autor: Manuel Daza Ramirez
#
# Propósito:
#   Establecer las importaciones necesarias, cargar credenciales de forma segura
#   y configurar el cliente del modelo de lenguaje.
#
# Dependencias requeridas (instalar con pip):
#   - python-dotenv: Carga variables de entorno desde archivo .env
#   - openai: SDK oficial para interactuar con la API de OpenAI
#   - pyyaml: Parser de archivos YAML para la ontología
# ==============================================================================

from dotenv import load_dotenv
import os

# Carga variables de entorno desde el archivo .env en el directorio raíz
# El archivo .env debe contener: OPENAI_API_KEY=sk-...
# IMPORTANTE: Nunca commitear el archivo .env a control de versiones
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Importaciones estándar para manipulación de datos
import json          # Parsing de respuestas JSON del modelo
import textwrap      # Utilidad para formateo de texto (no usada activamente aquí)
from dataclasses import dataclass  # Para estructuras de datos tipadas
from typing import List, Dict, Any  # Type hints para mejor documentación

# Cliente OpenAI - inicializa automáticamente con OPENAI_API_KEY del entorno
from openai import OpenAI
client = OpenAI()

# ==============================================================================
# CONFIGURACIÓN GLOBAL DEL MODELO
# ==============================================================================
# Modelo a utilizar para la clasificación.
# Opciones comunes:
#   - "gpt-4o": Modelo multimodal, buen balance costo/rendimiento
#   - "gpt-4-turbo": Alta capacidad, mayor costo
#   - "gpt-3.5-turbo": Más económico, menor precisión en tareas complejas
#
# NOTA: Cambiar este valor para probar diferentes modelos sin modificar código
# ==============================================================================
MODEL_NAME = "gpt-4o"

---
## 2. Preprocesamiento de texto

Los documentos testimoniales pueden venir de diversas fuentes (escaneos OCR, transcripciones, formularios digitales) y frecuentemente contienen:
- **Caracteres Unicode malformados**: Tildes descompuestas, espacios especiales
- **Espacios múltiples**: Resultado de copiar/pegar desde PDFs
- **Headers/footers institucionales**: Que no aportan al contenido testimonial

### Funciones implementadas:

| Función | Propósito |
|---------|----------|
| `normalize_unicode()` | Convierte caracteres a forma canónica NFC |
| `collapse_spaces()` | Reduce espacios múltiples a uno solo |
| `remove_headers_and_footers()` | Placeholder para patrones específicos UBPD |
| `preprocess_text()` | Pipeline completo de limpieza |

### Por qué es importante:
- El LLM procesa tokens; texto sucio puede desperdiciar tokens en ruido.
- La normalización mejora la consistencia de las clasificaciones.
- Prepara el texto para futuros procesamientos (embeddings, búsqueda, etc.).

In [ ]:
# ==============================================================================
# SECCIÓN 2: PREPROCESAMIENTO DE TEXTO
# ==============================================================================
# Propósito:
#   Limpiar y normalizar el texto de los documentos testimoniales antes de
#   enviarlo al modelo. Esto mejora la calidad de la clasificación y reduce
#   el consumo de tokens en texto irrelevante.
#
# El preprocesamiento es ligero intencionalmente:
#   - NO elimina puntuación (importante para el contexto)
#   - NO convierte a minúsculas (nombres propios son significativos)
#   - NO hace stemming/lemmatization (el LLM maneja esto internamente)
# ==============================================================================

import re           # Expresiones regulares para manipulación de texto
import unicodedata  # Normalización de caracteres Unicode


def normalize_unicode(text: str) -> str:
    """
    Normaliza el texto a forma canónica Unicode NFC.
    
    ¿Por qué NFC?
    -------------
    En Unicode, algunos caracteres pueden representarse de múltiples formas:
    - "á" puede ser un solo carácter (U+00E1) 
    - O la combinación de "a" (U+0061) + acento (U+0301)
    
    NFC (Canonical Decomposition, followed by Canonical Composition) asegura
    que caracteres equivalentes se representen de forma idéntica, evitando
    problemas de comparación y búsqueda.
    
    Args:
        text: Texto de entrada posiblemente con Unicode no normalizado
        
    Returns:
        Texto con caracteres Unicode en forma NFC
        
    Ejemplo:
        >>> normalize_unicode("café")  # Con tilde descompuesta
        'café'  # Con tilde compuesta
    """
    return unicodedata.normalize("NFC", text)


def collapse_spaces(text: str) -> str:
    """
    Reduce múltiples espacios en blanco consecutivos a un solo espacio.
    
    Caso de uso:
    ------------
    Documentos escaneados con OCR o copiados de PDFs frecuentemente tienen
    espacios múltiples, tabulaciones y saltos de línea irregulares que
    no aportan significado semántico.
    
    El patrón \s+ captura:
    - Espacios simples y múltiples
    - Tabulaciones (\t)
    - Saltos de línea (\n, \r)
    - Otros caracteres de espacio Unicode
    
    Args:
        text: Texto con posibles espacios múltiples
        
    Returns:
        Texto con espacios normalizados y sin espacios al inicio/final
        
    Ejemplo:
        >>> collapse_spaces("Hola    mundo  ")
        'Hola mundo'
    """
    return re.sub(r"\s+", " ", text).strip()


def remove_headers_and_footers(text: str) -> str:
    """
    Elimina encabezados y pies de página institucionales del documento.
    
    Estado actual: PLACEHOLDER
    --------------------------
    Esta función está preparada para incorporar patrones específicos de la UBPD
    una vez se identifiquen los formatos comunes de documentos.
    
    Patrones típicos a eliminar (ejemplos hipotéticos):
    - "UNIDAD DE BÚSQUEDA DE PERSONAS DADAS POR DESAPARECIDAS"
    - "Página X de Y"
    - Números de radicado
    - Firmas digitales y timestamps
    
    TODO: Implementar patrones reales basados en documentos UBPD
    
    Args:
        text: Texto del documento completo
        
    Returns:
        Texto sin elementos institucionales no relevantes para clasificación
    """
    # Ejemplo de implementación futura:
    # patterns = [
    #     r"^UBPD.*?\n",  # Header institucional
    #     r"Página \d+ de \d+",  # Numeración de páginas
    # ]
    # for pattern in patterns:
    #     text = re.sub(pattern, "", text, flags=re.MULTILINE)
    return text


def preprocess_text(text: str) -> str:
    """
    Pipeline completo de preprocesamiento para documentos testimoniales.
    
    Orden de operaciones:
    --------------------
    1. normalize_unicode: Garantiza consistencia de caracteres
    2. remove_headers_and_footers: Elimina ruido institucional
    3. collapse_spaces: Limpia espacios redundantes
    
    El orden importa:
    - La normalización Unicode debe ser primera para que los patrones regex
      en pasos posteriores funcionen correctamente.
    - collapse_spaces va al final para limpiar espacios que pudieron quedar
      tras remover headers/footers.
    
    Args:
        text: Texto crudo del documento testimonial
        
    Returns:
        Texto limpio y normalizado, listo para clasificación
        
    Ejemplo:
        >>> raw = "  Yo,  María,   cuento que en 1997...  "
        >>> preprocess_text(raw)
        'Yo, María, cuento que en 1997...'
    """
    text = normalize_unicode(text)
    text = remove_headers_and_footers(text)
    text = collapse_spaces(text)
    return text

---
## 3. Carga de la ontología UBPD

La **ontología** es el corazón del sistema de clasificación. Define el vocabulario controlado que garantiza:

1. **Consistencia**: Todos los documentos se clasifican con las mismas categorías.
2. **Interoperabilidad**: Los códigos pueden mapearse a sistemas externos (bases de datos, dashboards).
3. **Auditabilidad**: Las clasificaciones son trazables a categorías bien definidas.

### Estructura de la ontología YAML:

```yaml
tipo_documento:        # Naturaleza del documento
  TD0: "No testimonial"
  TD1: "Testimonio de víctima directa"
  TD2: "Testimonio de familiar"
  ...

tipo_hecho:            # Hechos victimizantes descritos
  TH0: "No aplica"
  TH1: "Desaparición forzada"
  TH2: "Homicidio"
  TH3: "Desplazamiento forzado"
  TH4: "Masacre"
  ...

actores:               # Presuntos responsables
  ACT0: "No identificado"
  ACT1: "Paramilitares"
  ACT2: "Guerrilla"
  ACT3: "Fuerza pública"
  ...

periodo:               # Época del conflicto
  PER0: "No identificado"
  PER1: "Antes de 1985"
  PER2: "1985-2000"
  ...

ruteo:                 # Área interna para seguimiento
  RU0: "Sin asignación"
  RU1: "Equipo de búsqueda prioritario"
  RU3: "Archivo general"
  ...
```

### ¿Por qué YAML?
- Legible para humanos (analistas pueden revisarlo sin programar)
- Fácil de versionar en Git
- Extensible sin cambiar código

In [ ]:
# ==============================================================================
# SECCIÓN 3: CARGA DE LA ONTOLOGÍA UBPD
# ==============================================================================
# Propósito:
#   Cargar el vocabulario controlado desde un archivo YAML externo.
#   La ontología define todas las categorías válidas para clasificación.
#
# Ventajas de externalizar la ontología:
#   - Modificable sin cambiar código (ej. agregar nuevos tipos de hecho)
#   - Versionable independientemente del código
#   - Reutilizable en otros sistemas (validadores, dashboards, reportes)
#
# Estructura esperada del YAML:
#   tipo_documento: {código: descripción}
#   tipo_hecho: {código: descripción}
#   territorio: [lista] o {código: descripción}
#   periodo: {código: descripción}
#   actores: {código: descripción}
#   ruteo: {código: descripción}
# ==============================================================================

import yaml  # Parser de archivos YAML


def load_ontology(path="../ontology_ubpd.yaml"):
    """
    Carga la ontología UBPD desde un archivo YAML.
    
    El archivo YAML define el vocabulario controlado para clasificación:
    - Códigos cortos (TD1, TH2, ACT3) para procesamiento
    - Descripciones legibles para interpretación humana
    
    Args:
        path: Ruta al archivo YAML de ontología.
              Por defecto busca en el directorio padre (../)
              
    Returns:
        dict: Diccionario con la estructura completa de la ontología
        
    Raises:
        FileNotFoundError: Si el archivo YAML no existe
        yaml.YAMLError: Si el archivo tiene sintaxis YAML inválida
        
    Ejemplo de uso:
        >>> ontology = load_ontology("ontology_ubpd.yaml")
        >>> print(ontology["tipo_hecho"]["TH1"])
        'Desaparición forzada'
    """
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)  # safe_load previene ejecución de código arbitrario


# Carga global de la ontología al iniciar el notebook
# Esta variable se usa en múltiples funciones posteriores
ONTOLOGY = load_ontology()

---
## 4. Serialización de ontología para el prompt

El LLM necesita conocer las categorías válidas para clasificar correctamente. Esta función convierte la ontología de Python a un formato de texto legible que se inyecta en el **system prompt**.

### Estrategia de prompt engineering:

Al incluir la lista completa de códigos válidos en el prompt, logramos:

1. **Restricción de vocabulario**: El modelo solo puede usar códigos que existen.
2. **Autodocumentación**: Las descripciones ayudan al modelo a elegir correctamente.
3. **Flexibilidad**: Cambios en la ontología se reflejan automáticamente.

### Formato de salida:

```
1. tipo_documento:
   - "TD0": "No testimonial"
   - "TD1": "Testimonio de víctima directa"
   ...

2. tipo_hecho:
   - "TH0": "No aplica"
   - "TH1": "Desaparición forzada"
   ...
```

Este formato es:
- Estructurado (numeración, indentación)
- No ambiguo (códigos entre comillas)
- Completo (todas las opciones visibles)

In [ ]:
# ==============================================================================
# SECCIÓN 4: SERIALIZACIÓN DE ONTOLOGÍA PARA EL PROMPT
# ==============================================================================
# Propósito:
#   Convertir la estructura de datos de la ontología (dict Python) a un
#   formato de texto legible que pueda incluirse en el prompt del modelo.
#
# Esta es una técnica clave de prompt engineering:
#   - El modelo ve TODAS las opciones válidas
#   - Cada código tiene su descripción para guiar la elección
#   - El formato estructurado facilita el parsing mental del modelo
# ==============================================================================

def ontology_to_prompt_text(ontology: dict) -> str:
    """
    Convierte la ontología cargada desde YAML a un bloque de texto
    estructurado para incluir en el system prompt del modelo.
    
    Formato de salida:
    -----------------
    Cada categoría se presenta con:
    - Número de sección (1, 2, 3...)
    - Nombre de la categoría
    - Lista de códigos con sus descripciones
    
    Decisiones de formato:
    - Comillas dobles alrededor de códigos: Evita ambigüedad
    - Indentación con espacios: Legibilidad visual
    - Líneas en blanco entre secciones: Separación clara
    
    Args:
        ontology: Diccionario con la estructura de la ontología
        
    Returns:
        str: Texto formateado listo para insertar en el prompt
        
    Nota sobre territorio:
        El campo territorio puede ser una lista simple ["Antioquia", "Chocó"]
        o un diccionario {código: descripción}. La función maneja ambos casos.
    """
    lines = []

    # ---------------------------------------------------------------------------
    # Tipo de documento: Clasifica la naturaleza del documento
    # TD0=No testimonial, TD1=Testimonio víctima, TD2=Testimonio familiar, etc.
    # ---------------------------------------------------------------------------
    lines.append("1. tipo_documento:")
    for code, label in ontology["tipo_documento"].items():
        lines.append(f'   - "{code}": "{label}"')

    # ---------------------------------------------------------------------------
    # Tipo de hecho: Hechos victimizantes mencionados en el documento
    # Puede haber múltiples hechos en un mismo testimonio
    # ---------------------------------------------------------------------------
    lines.append("\n2. tipo_hecho:")
    for code, label in ontology["tipo_hecho"].items():
        lines.append(f'   - "{code}": "{label}"')

    # ---------------------------------------------------------------------------
    # Territorio: Ubicación geográfica de los hechos
    # Formato flexible: puede ser lista o diccionario según el YAML
    # ---------------------------------------------------------------------------
    if "territorio" in ontology:
        lines.append("\n3. territorio:")
        if isinstance(ontology["territorio"], list):
            # Caso: lista simple de departamentos
            for item in ontology["territorio"]:
                lines.append(f'   - "{item}"')
        else:
            # Caso: diccionario código → descripción
            for k, v in ontology["territorio"].items():
                lines.append(f'   - "{k}": "{v}"')

    # ---------------------------------------------------------------------------
    # Periodo: Época del conflicto en que ocurrieron los hechos
    # Importante para contextualización histórica
    # ---------------------------------------------------------------------------
    lines.append("\n4. periodo:")
    for code, label in ontology["periodo"].items():
        lines.append(f'   - "{code}": "{label}"')

    # ---------------------------------------------------------------------------
    # Actores: Presuntos responsables de los hechos
    # Categoría sensible que requiere precisión
    # ---------------------------------------------------------------------------
    lines.append("\n5. actores:")
    for code, label in ontology["actores"].items():
        lines.append(f'   - "{code}": "{label}"')

    # ---------------------------------------------------------------------------
    # Ruteo: Área interna de la UBPD para seguimiento del caso
    # Determina el flujo de trabajo posterior
    # ---------------------------------------------------------------------------
    lines.append("\n6. ruteo:")
    for code, label in ontology["ruteo"].items():
        lines.append(f'   - "{code}": "{label}"')

    return "\n".join(lines)


# Pre-calcular el texto de ontología para reutilizar en el prompt
# Esto evita regenerarlo en cada llamada de clasificación
ONTOLOGY_PROMPT = ontology_to_prompt_text(ONTOLOGY)

---
## 5. Definición del System Prompt

El **system prompt** es la instrucción principal que define el comportamiento del modelo. Es el componente más crítico del sistema.

### Principios de diseño aplicados:

| Principio | Implementación |
|-----------|---------------|
| **Rol claro** | "Eres un clasificador" - establece la identidad |
| **Restricciones explícitas** | Lista cerrada de códigos válidos |
| **Formato obligatorio** | Template JSON exacto a seguir |
| **Reglas de negocio** | "Si TD0 entonces RU0" |
| **Auto-verificación** | Instrucciones internas para que el modelo valide |
| **Salida limpia** | "SOLO el JSON, sin texto adicional" |

### Anatomía del prompt:

```
┌─────────────────────────────────────────┐
│  1. Definición de rol                   │
├─────────────────────────────────────────┤
│  2. Ontología completa (códigos válidos)│
├─────────────────────────────────────────┤
│  3. Reglas de negocio                   │
├─────────────────────────────────────────┤
│  4. Formato JSON obligatorio            │
├─────────────────────────────────────────┤
│  5. Instrucciones de auto-verificación  │
└─────────────────────────────────────────┘
```

### Reglas de negocio implementadas:

1. **TD0 → RU0**: Documentos no testimoniales no se enrutan a equipos de búsqueda.
2. **Actor por defecto**: Si no hay actor identificado, usar `ACT0`.
3. **Territorio por defecto**: Si no hay ubicación, usar `["No identificado"]`.

In [ ]:
# ==============================================================================
# SECCIÓN 5: DEFINICIÓN DEL SYSTEM PROMPT
# ==============================================================================
# Propósito:
#   Definir las instrucciones principales que guían el comportamiento del modelo.
#   Este prompt establece:
#     - El rol del modelo (clasificador)
#     - Los vocabularios válidos (ontología)
#     - Las reglas de negocio (ej. TD0 → RU0)
#     - El formato de salida (JSON estricto)
#     - Instrucciones de auto-verificación
#
# Técnicas de prompt engineering utilizadas:
#   1. Role prompting: "Eres un clasificador"
#   2. Constraint prompting: Listas cerradas de códigos
#   3. Format specification: Template JSON exacto
#   4. Chain-of-thought implícito: "INSTRUCCIONES INTERNAS"
#   5. Self-verification: "VERIFICA tú mismo"
# ==============================================================================

SYSTEM_PROMPT = f"""
Eres un clasificador. Tu única tarea es devolver un JSON válido con etiquetas que usen EXCLUSIVAMENTE los códigos de esta ontología:

CÓDIGOS VÁLIDOS (listas cerradas):
{ONTOLOGY_PROMPT}

Reglas clave:
- tipo_documento ∈ lista tipo_documento.
- tipo_hecho ⊆ lista tipo_hecho (puede ser lista vacía).
- periodo ∈ lista periodo.
- actores ∈ lista actores (si no aparece ningún actor, usa ["ACT0"]).
- ruteo ∈ lista ruteo.
- territorio es una lista de nombres de departamentos de Colombia o ["No identificado"].
- highlights es una lista de frases textuales del documento (puede estar vacía).
- Si tipo_documento = "TD0" entonces ruteo = "RU0".

Formato JSON OBLIGATORIO (ni más ni menos campos):

{{
  "tipo_documento": "TDx",
  "tipo_hecho": ["THx", "..."],
  "territorio": ["Nombre departamento o 'No identificado'", "..."],
  "periodo": "PERx",
  "actores": ["ACTx", "..."],
  "ruteo": "RUx",
  "highlights": ["frase 1", "frase 2", ...]
}}

INSTRUCCIONES INTERNAS (no las muestres):
1) Lee el documento y decide las etiquetas adecuadas.
2) Construye mentalmente el JSON.
3) VERIFICA tú mismo:
   - ¿Todos los campos existen y solo esos?
   - ¿Todos los códigos pertenecen a las listas válidas?
   - ¿Se cumple la regla: si TD0 → RU0?
   - ¿Territorio es una lista y no está vacía (si no hay info, ["No identificado"])?
   - ¿highlights es una lista (posiblemente vacía)?
4) Si encuentras algún error, corrígelo antes de responder.
5) Responde SOLO el JSON final, sin texto adicional.
""".strip()

# Nota sobre el uso de f-string con dobles llaves {{}}:
# Las dobles llaves se escapan a llaves simples en el string final.
# Esto es necesario porque {ONTOLOGY_PROMPT} es una variable f-string,
# pero las llaves del template JSON deben permanecer literales.

---
## 6. Template del User Prompt con Few-Shot Examples

El **user prompt** contiene el documento a clasificar junto con **ejemplos demostrativos** (few-shot learning).

### ¿Por qué few-shot prompting?

Los ejemplos ayudan al modelo a:
1. **Entender el formato esperado**: JSON concreto, no abstracto.
2. **Calibrar el nivel de detalle**: Qué tan específicos deben ser los highlights.
3. **Manejar casos límite**: El Ejemplo 2 muestra qué hacer con documentos no testimoniales.

### Estructura del template:

```
┌─────────────────────────────────────────┐
│  Ejemplo 1: Testimonio de desaparición  │
│  - Entrada (texto)                      │
│  - Salida esperada (JSON)               │
├─────────────────────────────────────────┤
│  Ejemplo 2: Documento no testimonial    │
│  - Entrada (texto)                      │
│  - Salida esperada (JSON)               │
├─────────────────────────────────────────┤
│  Documento a clasificar                 │
│  {{DOCUMENTO}}                          │
├─────────────────────────────────────────┤
│  Recordatorio de reglas                 │
└─────────────────────────────────────────┘
```

### Selección de ejemplos:

| Ejemplo | Propósito | Características |
|---------|-----------|----------------|
| **Ejemplo 1** | Caso típico | Desaparición forzada, territorio específico, actor identificado |
| **Ejemplo 2** | Caso negativo | Documento administrativo, muestra TD0 y valores vacíos |

In [ ]:
# ==============================================================================
# SECCIÓN 6: TEMPLATE DEL USER PROMPT CON EJEMPLOS (FEW-SHOT)
# ==============================================================================
# Propósito:
#   Definir el template que contiene los ejemplos demostrativos y el
#   marcador de posición para el documento a clasificar.
#
# Few-shot learning:
#   Al mostrar ejemplos concretos de entrada→salida, el modelo aprende:
#   - El formato exacto del JSON esperado
#   - Cómo mapear contenido textual a códigos de ontología
#   - Cómo manejar casos donde información no está disponible
#
# Selección de ejemplos:
#   Ejemplo 1: Testimonio típico con información completa
#     - Tipo: Testimonio de víctima (TD1)
#     - Hechos: Desaparición forzada (TH1), desplazamiento (TH3)
#     - Ubicación específica: San Carlos, Antioquia
#     - Actor: Guerrilla (ACT2)
#     - Ruteo: Equipo prioritario (RU1)
#
#   Ejemplo 2: Documento NO testimonial (contraejemplo)
#     - Tipo: No testimonial (TD0)
#     - Demuestra uso de valores por defecto
#     - Ruteo obligatorio RU0 por regla de negocio
# ==============================================================================

USER_TEMPLATE = """
Ejemplo 1 (entrada):
"Yo, María, cuento que en 1997, en el municipio de San Carlos, Antioquia, hombres armados de la guerrilla se llevaron a mi esposo. Nos tocó salir para Medellín."

Ejemplo 1 (salida):
{
  "tipo_documento": "TD1",
  "tipo_hecho": ["TH1","TH3"],
  "territorio": ["Antioquia"],
  "periodo": "PER2",
  "actores": ["ACT2"],
  "ruteo": "RU1",
  "highlights": [
    "1997, en el municipio de San Carlos, Antioquia",
    "se llevaron a mi esposo"
  ]
}

Ejemplo 2 (entrada):
"Oficio No. 123 de 2020. Remito informe técnico sobre la migración de datos."

Ejemplo 2 (salida):
{
  "tipo_documento": "TD0",
  "tipo_hecho": [],
  "territorio": ["No identificado"],
  "periodo": "PER5",
  "actores": ["ACT0"],
  "ruteo": "RU0",
  "highlights": []
}

Ahora clasifica este documento siguiendo estrictamente el formato de los ejemplos y las reglas del sistema:

DOCUMENTO:
{{DOCUMENTO}}

Recuerda:
- Usa únicamente los códigos válidos.
- Verifica internamente que el JSON cumple todas las reglas.
- Responde SOLO el JSON, sin comentarios adicionales.
""".strip()


def build_user_prompt(text: str) -> str:
    """
    Construye el prompt de usuario insertando el documento a clasificar.
    
    El marcador {{DOCUMENTO}} se reemplaza con el texto limpio del documento.
    Se usa doble llave para evitar conflictos con f-strings de Python.
    
    Args:
        text: Texto del documento ya preprocesado
        
    Returns:
        str: Prompt completo con ejemplos y documento a clasificar
        
    Ejemplo:
        >>> prompt = build_user_prompt("Mi hermano desapareció en 2003...")
        >>> "Mi hermano desapareció" in prompt
        True
    """
    return USER_TEMPLATE.replace("{{DOCUMENTO}}", text.strip())

---
## 7. Cliente LLM y función de llamada

Esta función encapsula la comunicación con el modelo de lenguaje.

### Parámetros clave:

| Parámetro | Valor | Justificación |
|-----------|-------|---------------|
| `model` | `MODEL_NAME` | Configurable globalmente |
| `temperature` | `0.0` | **Determinismo**: Misma entrada → misma salida |
| `messages` | system + user | Separación de instrucciones y datos |

### ¿Por qué temperature=0.0?

Para tareas de **clasificación** donde necesitamos:
- **Reproducibilidad**: El mismo documento debe clasificarse igual siempre.
- **Auditabilidad**: Resultados deben ser explicables y consistentes.
- **Confianza**: Reducir variabilidad en decisiones importantes.

### Estructura de mensajes:

```python
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},  # Instrucciones permanentes
    {"role": "user", "content": user_prompt}       # Documento + ejemplos
]
```

El rol `system` tiene mayor peso que `user` en la mayoría de los modelos, lo que refuerza las restricciones de la ontología.

In [ ]:
# ==============================================================================
# SECCIÓN 7: CLIENTE LLM Y FUNCIÓN DE LLAMADA
# ==============================================================================
# Propósito:
#   Encapsular la comunicación con el modelo de lenguaje en una función
#   reutilizable que maneja la estructura de mensajes y parámetros.
#
# Diseño:
#   - Recibe prompts como parámetros (no hardcodeados)
#   - Usa temperature=0.0 para clasificación determinística
#   - Retorna texto crudo para procesamiento posterior
#
# Extensibilidad:
#   Esta función puede modificarse para:
#   - Usar otros proveedores (Azure OpenAI, Anthropic, local)
#   - Agregar retry logic para errores de red
#   - Implementar rate limiting
#   - Agregar logging de llamadas
# ==============================================================================

def call_llm(system_prompt: str, user_prompt: str) -> str:
    """
    Realiza una llamada al modelo de lenguaje y retorna la respuesta.
    
    Estructura de la llamada:
    ------------------------
    - system_prompt: Instrucciones de alto nivel (rol, reglas, formato)
    - user_prompt: Documento a clasificar con ejemplos few-shot
    
    Configuración:
    - model: Especificado por MODEL_NAME global
    - temperature: 0.0 para máximo determinismo
      (el modelo elegirá siempre el token más probable)
    
    Args:
        system_prompt: Instrucciones del sistema con ontología y reglas
        user_prompt: Documento a clasificar envuelto en template con ejemplos
        
    Returns:
        str: Texto crudo de la respuesta del modelo (idealmente JSON puro)
        
    Raises:
        openai.APIError: Si hay problemas de comunicación con la API
        openai.RateLimitError: Si se excede el límite de solicitudes
        
    Nota sobre el código muerto:
        El `raise NotImplementedError` después del return es código
        inalcanzable, probablemente residuo de desarrollo. En producción
        debería eliminarse.
    """
    # Construir la solicitud al modelo
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            # El mensaje "system" establece el contexto persistente
            # El modelo lo trata como instrucciones de mayor autoridad
            {"role": "system", "content": system_prompt},
            
            # El mensaje "user" contiene el documento específico a clasificar
            {"role": "user", "content": user_prompt},
        ],
        # temperature=0.0: Elimina aleatoriedad en la selección de tokens
        # Esto hace que la clasificación sea reproducible
        temperature=0.0,
    )
    
    # Extraer el contenido de texto de la primera opción de respuesta
    # En configuraciones típicas solo hay una opción (n=1 por defecto)
    return response.choices[0].message.content

    # NOTA: Este código es inalcanzable y debería eliminarse en producción
    raise NotImplementedError("Implementa aquí la llamada real al LLM.")

---
## 8. Extracción segura de JSON

Aunque el prompt pide "SOLO el JSON", los modelos a veces agregan texto adicional. Estas funciones manejan esa variabilidad.

### Casos que maneja `extract_json_block`:

```
Caso 1 (ideal): Solo JSON
{"tipo_documento": "TD1", ...}

Caso 2: JSON con texto antes
Aquí está la clasificación:
{"tipo_documento": "TD1", ...}

Caso 3: JSON con texto después
{"tipo_documento": "TD1", ...}
Espero que esto sea útil.

Caso 4: JSON envuelto en markdown
```json
{"tipo_documento": "TD1", ...}
```
```

### Estrategia:
1. Buscar la primera `{` (inicio del JSON)
2. Buscar la última `}` (fin del JSON)
3. Extraer la subcadena entre ambas posiciones

**Limitación**: Si hay múltiples objetos JSON, solo extrae el más externo. Para nuestro caso de uso (un solo JSON de clasificación) esto es suficiente.

In [ ]:
# ==============================================================================
# SECCIÓN 8: EXTRACCIÓN SEGURA DE JSON
# ==============================================================================
# Propósito:
#   Extraer de forma robusta el JSON de clasificación de la respuesta del modelo,
#   manejando casos donde el modelo incluye texto adicional.
#
# Problema:
#   Aunque el prompt pide "SOLO el JSON", los modelos a veces agregan:
#   - Texto introductorio: "Aquí está la clasificación:"
#   - Bloques de código markdown: ```json ... ```
#   - Explicaciones finales: "Nota: el periodo es..."
#
# Solución:
#   Buscar los delimitadores del JSON ({ y }) y extraer solo ese bloque.
# ==============================================================================

def extract_json_block(text: str) -> str:
    """
    Extrae el primer bloque JSON del texto de respuesta del modelo.
    
    Algoritmo:
    ---------
    1. Busca la primera aparición de '{' (inicio del objeto JSON)
    2. Busca la última aparición de '}' (fin del objeto JSON)
    3. Retorna la subcadena entre ambas posiciones (inclusive)
    
    Este enfoque es simple pero efectivo para el caso de uso:
    - Esperamos UN SOLO objeto JSON por respuesta
    - El JSON de clasificación siempre es un objeto (no array)
    - No hay JSON anidado que pudiera confundir los delimitadores
    
    Args:
        text: Respuesta cruda del modelo, posiblemente con texto adicional
        
    Returns:
        str: Bloque JSON extraído como string
        
    Raises:
        ValueError: Si no se encuentra un JSON válido (sin { o sin })
        
    Ejemplos:
        >>> extract_json_block('Resultado: {"a": 1}')
        '{"a": 1}'
        >>> extract_json_block('```json\n{"a": 1}\n```')
        '{"a": 1}'
    """
    # Encontrar posición del primer carácter '{'
    start = text.find("{")
    
    # Encontrar posición del último carácter '}'
    # Usamos rfind (reverse find) para encontrar desde el final
    end = text.rfind("}")
    
    # Validar que ambos delimitadores existan y estén en orden correcto
    if start == -1 or end == -1 or end <= start:
        raise ValueError("No se encontró un JSON válido en la respuesta del modelo.")
    
    # Extraer subcadena incluyendo ambos delimitadores (+1 para incluir '}')
    return text[start:end+1]


def parse_model_response(raw_response: str) -> Dict[str, Any]:
    """
    Parsea la respuesta del modelo a un diccionario Python.
    
    Pipeline:
    1. extract_json_block: Aisla el JSON del texto circundante
    2. json.loads: Convierte string JSON a dict Python
    
    Args:
        raw_response: Respuesta cruda del modelo
        
    Returns:
        dict: Diccionario con la clasificación parseada
        
    Raises:
        ValueError: Si no hay JSON en la respuesta
        json.JSONDecodeError: Si el JSON está malformado
        
    Ejemplo:
        >>> parse_model_response('{"tipo_documento": "TD1"}')
        {'tipo_documento': 'TD1'}
    """
    json_str = extract_json_block(raw_response)
    return json.loads(json_str)

---
## 9. Helpers de validación y normalización

Aunque el LLM está instruido para usar solo códigos válidos, puede cometer errores. Estas funciones proporcionan una **red de seguridad** que garantiza que la salida final siempre cumpla con la ontología.

### Filosofía: "Trust but verify"

- **Confiar**: El prompt está diseñado para producir salidas correctas.
- **Verificar**: La validación post-hoc corrige errores silenciosamente.

### Funciones de corrección:

| Función | Entrada | Comportamiento |
|---------|---------|----------------|
| `fix_single_label` | String | Devuelve default si código inválido |
| `fix_multi_labels` | Lista | Filtra códigos inválidos |
| `fix_territorio` | Lista | Normaliza y asegura lista no vacía |

### Valores por defecto:

| Campo | Default | Significado |
|-------|---------|-------------|
| `tipo_documento` | TD0 | No testimonial |
| `periodo` | PER0 | No identificado |
| `ruteo` | RU0 | Sin asignación |
| `territorio` | ["No identificado"] | Ubicación desconocida |

In [ ]:
# ==============================================================================
# SECCIÓN 9: HELPERS DE VALIDACIÓN Y NORMALIZACIÓN
# ==============================================================================
# Propósito:
#   Funciones auxiliares para corregir y normalizar etiquetas de clasificación.
#   Actúan como red de seguridad cuando el modelo produce códigos inválidos.
#
# Principio de diseño:
#   - Nunca fallar silenciosamente
#   - Siempre producir una salida válida (usando defaults cuando sea necesario)
#   - Preservar la mayor cantidad de información correcta posible
#
# Estas funciones NO modifican el input original, retornan valores corregidos.
# ==============================================================================

def fix_single_label(value: str, valid_codes, default: str) -> str:
    """
    Valida y corrige una etiqueta de valor único.
    
    Usado para campos que aceptan UN SOLO valor:
    - tipo_documento
    - periodo
    - ruteo
    
    Lógica:
    - Si el valor está en la lista de códigos válidos → lo retorna tal cual
    - Si el valor es inválido → retorna el default
    
    Args:
        value: Código retornado por el modelo
        valid_codes: Iterable de códigos válidos (ej. ONTOLOGY["periodo"].keys())
        default: Valor a usar si el código es inválido
        
    Returns:
        str: Código válido (original o default)
        
    Ejemplo:
        >>> fix_single_label("TD1", ["TD0", "TD1", "TD2"], "TD0")
        'TD1'  # Válido, retorna tal cual
        >>> fix_single_label("TDX", ["TD0", "TD1", "TD2"], "TD0")
        'TD0'  # Inválido, retorna default
    """
    if value in valid_codes:
        return value
    return default


def fix_multi_labels(values, valid_codes) -> List[str]:
    """
    Valida y filtra una lista de etiquetas múltiples.
    
    Usado para campos que aceptan MÚLTIPLES valores:
    - tipo_hecho (puede haber varios hechos victimizantes)
    - actores (puede haber varios actores involucrados)
    
    Lógica:
    - Si la entrada no es lista → retorna lista vacía
    - Filtra solo los valores que están en valid_codes
    - Códigos inválidos se descartan silenciosamente
    
    Args:
        values: Lista de códigos retornada por el modelo (o None/otro tipo)
        valid_codes: Iterable de códigos válidos
        
    Returns:
        List[str]: Lista filtrada con solo códigos válidos
        
    Ejemplo:
        >>> fix_multi_labels(["TH1", "TH2", "INVALID"], ["TH1", "TH2", "TH3"])
        ['TH1', 'TH2']  # INVALID se descarta
        >>> fix_multi_labels("TH1", ["TH1", "TH2"])  # String en vez de lista
        []  # Retorna lista vacía
    """
    if not isinstance(values, list):
        return []
    return [v for v in values if v in valid_codes]


def fix_territorio(values: List[str]) -> List[str]:
    """
    Normaliza y valida la lista de territorios.
    
    El territorio es un caso especial:
    - No usa códigos predefinidos (son nombres de departamentos)
    - Debe ser siempre una lista no vacía
    - Si está vacío, usar ["No identificado"]
    
    Normalizaciones:
    - Elimina espacios al inicio/final de cada valor
    - Elimina duplicados (usando set)
    
    Args:
        values: Lista de nombres de departamentos/territorios
        
    Returns:
        List[str]: Lista normalizada y deduplicada, o ["No identificado"]
        
    Ejemplo:
        >>> fix_territorio([" Antioquia ", "Chocó", "Antioquia"])
        ['Antioquia', 'Chocó']  # Normalizado y deduplicado
        >>> fix_territorio([])  # Lista vacía
        ['No identificado']  # Default
    """
    # Validar que sea lista no vacía
    if not isinstance(values, list) or len(values) == 0:
        return ["No identificado"]
    
    # Normalizar: strip + deduplicar
    # El set elimina duplicados, luego convertimos a lista
    return list({v.strip() for v in values})

---
## 10. Cálculo de prioridad para enrutamiento

El **priority_score** es un valor numérico [0.0, 1.0] que ayuda a priorizar casos para atención.

### Lógica de puntuación:

| Condición | Puntos | Justificación |
|-----------|--------|---------------|
| TH1 (Desaparición forzada) | +0.4 | Mandato principal de la UBPD |
| TH4 (Masacre) | +0.2 | Alto impacto, múltiples víctimas |
| RU1 (Equipo prioritario) | +0.3 | Ya identificado como urgente |
| RU3 (Archivo general) | +0.1 | Relevancia menor pero presente |

### Interpretación del score:

```
0.0 - 0.3: Prioridad baja (documentos administrativos, casos cerrados)
0.4 - 0.6: Prioridad media (testimonios con información parcial)
0.7 - 1.0: Prioridad alta (desapariciones activas, nuevas pistas)
```

### Notas de diseño:

- Los pesos son **configurables** y deberían ajustarse con expertos del dominio.
- El `min(score, 1.0)` garantiza que el score nunca exceda 1.0.
- En producción, este cálculo podría evolucionar a un modelo ML más sofisticado.

In [ ]:
# ==============================================================================
# SECCIÓN 10: CÁLCULO DE PRIORIDAD PARA ENRUTAMIENTO
# ==============================================================================
# Propósito:
#   Calcular un score de prioridad [0.0, 1.0] basado en las etiquetas
#   de clasificación. Este score ayuda a ordenar casos para atención.
#
# Contexto de negocio:
#   La UBPD tiene recursos limitados y debe priorizar casos donde:
#   - Hay desaparición forzada (su mandato principal)
#   - Hay múltiples víctimas (masacres)
#   - El documento ya fue enrutado a equipos especializados
#
# Los pesos actuales son un punto de partida; deberían calibrarse
# con datos históricos y expertos del dominio.
# ==============================================================================

def compute_priority(pred: Dict[str, Any]) -> float:
    """
    Calcula un score de prioridad basado en la clasificación.
    
    Sistema de puntos:
    -----------------
    El score es la suma de puntos por condiciones cumplidas:
    
    +0.4 puntos si tipo_hecho incluye TH1 (Desaparición forzada)
         Razón: Es el mandato principal de la UBPD
    
    +0.2 puntos si tipo_hecho incluye TH4 (Masacre)
         Razón: Alto impacto, afecta múltiples familias
    
    +0.3 puntos si ruteo es RU1 (Equipo de búsqueda prioritario)
         Razón: Ya identificado como caso urgente
    
    +0.1 puntos si ruteo es RU3 (Archivo general)
         Razón: Tiene relevancia pero menor urgencia
    
    Ejemplos de scores:
    - Desaparición + Equipo prioritario: 0.4 + 0.3 = 0.7
    - Masacre + Desaparición + Prioritario: 0.4 + 0.2 + 0.3 = 0.9
    - Documento no testimonial: 0.0
    
    Args:
        pred: Diccionario con la clasificación del documento
        
    Returns:
        float: Score de prioridad entre 0.0 y 1.0
        
    Nota: El score se limita a 1.0 máximo aunque la suma pudiera excederlo.
    """
    score = 0.0
    
    # Extraer campos relevantes con defaults seguros
    hechos = set(pred.get("tipo_hecho", []))  # Convertir a set para búsqueda O(1)
    ruteo = pred.get("ruteo")

    # -------------------------------------------------------------------------
    # Puntos por tipo de hecho
    # -------------------------------------------------------------------------
    if "TH1" in hechos:  # Desaparición forzada
        score += 0.4
        # Esta es la prioridad más alta porque es el mandato central de la UBPD:
        # buscar personas desaparecidas en el contexto del conflicto armado.
    
    if "TH4" in hechos:  # Masacre
        score += 0.2
        # Las masacres implican múltiples víctimas y frecuentemente
        # están relacionadas con desapariciones en fosas comunes.

    # -------------------------------------------------------------------------
    # Puntos por ruteo asignado
    # -------------------------------------------------------------------------
    if ruteo == "RU1":  # Equipo de búsqueda prioritario
        score += 0.3
        # El clasificador o una regla previa ya identificó este caso
        # como merecedor de atención especializada.
    
    if ruteo == "RU3":  # Archivo general
        score += 0.1
        # Documentos que van al archivo tienen menor urgencia
        # pero siguen siendo relevantes para investigación histórica.

    # Limitar el score máximo a 1.0
    # Esto normaliza el score independientemente de cuántas condiciones se cumplan
    return min(score, 1.0)

---
## 11. Validación completa de la predicción

Esta función integra todas las validaciones individuales y aplica **reglas de negocio** adicionales.

### Pipeline de validación:

```
Predicción cruda del modelo
         │
         ▼
┌─────────────────────────────────────┐
│ 1. Validar tipo_documento           │
│ 2. Validar tipo_hecho               │
│ 3. Validar periodo                  │
│ 4. Validar actores                  │
│ 5. Validar ruteo                    │
│ 6. Normalizar territorio            │
│ 7. Aplicar regla TD0 → RU0          │
│ 8. Garantizar highlights es lista   │
│ 9. Calcular priority_score          │
└─────────────────────────────────────┘
         │
         ▼
Predicción validada y enriquecida
```

### Regla de negocio: TD0 → RU0

Si el documento es **no testimonial** (TD0), automáticamente se asigna al ruteo **sin asignación** (RU0). Esto evita que documentos administrativos lleguen a equipos de búsqueda.

### Enriquecimiento:

La función añade `priority_score` que no viene del modelo pero es calculado localmente.

In [ ]:
# ==============================================================================
# SECCIÓN 11: VALIDACIÓN COMPLETA DE LA PREDICCIÓN
# ==============================================================================
# Propósito:
#   Aplicar todas las validaciones y reglas de negocio a la predicción
#   del modelo antes de retornar el resultado final.
#
# Esta función es el "firewall" final que garantiza:
#   - Todos los códigos son válidos según la ontología
#   - Las reglas de negocio se cumplen (ej. TD0 → RU0)
#   - Los campos tienen los tipos correctos (listas, strings)
#   - Se añade el campo calculado priority_score
#
# IMPORTANTE: Esta función MODIFICA el diccionario in-place.
# En producción, considerar hacer una copia para inmutabilidad.
# ==============================================================================

def validate_and_fix(pred: Dict[str, Any]) -> Dict[str, Any]:
    """
    Valida y corrige la predicción del modelo aplicando la ontología
    y las reglas de negocio.
    
    Proceso:
    -------
    1. Validar cada campo contra su lista de códigos válidos
    2. Aplicar valores por defecto donde sea necesario
    3. Ejecutar reglas de negocio (ej. TD0 → RU0)
    4. Garantizar tipos de datos correctos
    5. Calcular y añadir priority_score
    
    Args:
        pred: Diccionario con la predicción cruda del modelo
        
    Returns:
        Dict[str, Any]: Predicción validada, corregida y enriquecida
        
    Side effects:
        Modifica el diccionario pred in-place (además de retornarlo)
        
    Ejemplo:
        >>> raw = {"tipo_documento": "INVALID", "tipo_hecho": ["TH1", "BAD"]}
        >>> fixed = validate_and_fix(raw)
        >>> fixed["tipo_documento"]
        'TD0'  # Corregido a default
        >>> fixed["tipo_hecho"]
        ['TH1']  # 'BAD' fue filtrado
    """
    
    # -------------------------------------------------------------------------
    # Validación de campos de valor único
    # Cada campo se valida contra su lista de códigos en la ontología
    # Si el código es inválido, se usa el default especificado
    # -------------------------------------------------------------------------
    
    pred["tipo_documento"] = fix_single_label(
        pred.get("tipo_documento"),           # Valor del modelo
        ONTOLOGY["tipo_documento"].keys(),    # Códigos válidos: TD0, TD1, TD2...
        default="TD0"                         # Default: No testimonial
    )

    pred["periodo"] = fix_single_label(
        pred.get("periodo"),
        ONTOLOGY["periodo"].keys(),           # Códigos válidos: PER0, PER1, PER2...
        default="PER0"                        # Default: No identificado
    )

    pred["ruteo"] = fix_single_label(
        pred.get("ruteo"),
        ONTOLOGY["ruteo"].keys(),             # Códigos válidos: RU0, RU1, RU2...
        default="RU0"                         # Default: Sin asignación
    )

    # -------------------------------------------------------------------------
    # Validación de campos multi-valor
    # Filtra códigos inválidos, preservando los válidos
    # -------------------------------------------------------------------------
    
    pred["tipo_hecho"] = fix_multi_labels(
        pred.get("tipo_hecho", []),           # Lista de hechos del modelo
        ONTOLOGY["tipo_hecho"].keys()         # Códigos válidos: TH0, TH1, TH2...
    )

    pred["actores"] = fix_multi_labels(
        pred.get("actores", []),              # Lista de actores del modelo
        ONTOLOGY["actores"].keys()            # Códigos válidos: ACT0, ACT1, ACT2...
    )

    # -------------------------------------------------------------------------
    # Normalización de territorio
    # No usa códigos fijos, pero debe ser lista no vacía
    # -------------------------------------------------------------------------
    
    pred["territorio"] = fix_territorio(pred.get("territorio", []))

    # -------------------------------------------------------------------------
    # REGLA DE NEGOCIO: TD0 → RU0
    # -------------------------------------------------------------------------
    # Si el documento no es testimonial (TD0), no debe enrutarse a ningún
    # equipo de búsqueda. Se fuerza el ruteo a RU0 (Sin asignación).
    #
    # Justificación: Documentos administrativos, oficios, informes técnicos,
    # etc. no contienen información de búsqueda y procesarlos desperdiciaría
    # recursos de los equipos especializados.
    # -------------------------------------------------------------------------
    
    if pred["tipo_documento"] == "TD0":
        pred["ruteo"] = "RU0"

    # -------------------------------------------------------------------------
    # Garantizar que highlights sea siempre una lista
    # -------------------------------------------------------------------------
    # El modelo podría retornar None, un string, u otro tipo.
    # Forzamos a lista vacía si no es una lista válida.
    
    if not isinstance(pred.get("highlights"), list):
        pred["highlights"] = []

    # -------------------------------------------------------------------------
    # Cálculo del score de prioridad
    # -------------------------------------------------------------------------
    # Este campo NO viene del modelo; se calcula localmente basándose
    # en las etiquetas validadas.
    
    pred["priority_score"] = compute_priority(pred)
    
    return pred

---
## 12. Función principal: classify_document

Esta es la **API pública** del clasificador. Orquesta todo el pipeline desde texto crudo hasta clasificación validada.

### Pipeline completo:

```
Texto crudo del documento
         │
         ▼
┌─────────────────────────────┐
│   1. preprocess_text()      │  Limpieza y normalización
└─────────────────────────────┘
         │
         ▼
┌─────────────────────────────┐
│   2. build_user_prompt()    │  Insertar documento en template
└─────────────────────────────┘
         │
         ▼
┌─────────────────────────────┐
│   3. call_llm()             │  Llamada al modelo
└─────────────────────────────┘
         │
         ▼
┌─────────────────────────────┐
│   4. parse_model_response() │  Extraer y parsear JSON
└─────────────────────────────┘
         │
         ▼
┌─────────────────────────────┐
│   5. validate_and_fix()     │  Validar y enriquecer
└─────────────────────────────┘
         │
         ▼
Clasificación final validada
```

### Uso:

```python
resultado = classify_document("Mi hermano desapareció en 2003...")
print(resultado["tipo_hecho"])      # ['TH1']
print(resultado["priority_score"])  # 0.7
```

In [ ]:
# ==============================================================================
# SECCIÓN 12: FUNCIÓN PRINCIPAL DE CLASIFICACIÓN
# ==============================================================================
# Autor: Manuel Daza Ramirez
#
# Propósito:
#   Exponer una única función de alto nivel que encapsula todo el pipeline
#   de clasificación de documentos testimoniales.
#
# Esta es la API pública del clasificador:
#   - Input: Texto crudo del documento
#   - Output: Diccionario con clasificación validada y score de prioridad
#
# Composición del pipeline:
#   1. preprocess_text → Limpieza de texto
#   2. build_user_prompt → Construcción del prompt
#   3. call_llm → Llamada al modelo
#   4. parse_model_response → Extracción de JSON
#   5. validate_and_fix → Validación y enriquecimiento
# ==============================================================================

def classify_document(text: str) -> Dict[str, Any]:
    """
    Clasifica un documento testimonial usando LLM y ontología controlada.
    
    Esta función es el punto de entrada principal del clasificador.
    Toma texto crudo y retorna una clasificación estructurada con:
    - Tipo de documento (testimonial/no testimonial)
    - Hechos victimizantes identificados
    - Territorio geográfico
    - Período temporal
    - Actores involucrados
    - Ruteo interno sugerido
    - Fragmentos destacados (highlights)
    - Score de prioridad calculado
    
    Pipeline:
    ---------
    text → preprocess → prompt → LLM → parse → validate → resultado
    
    Args:
        text: Texto del documento testimonial (crudo, sin preprocesar)
        
    Returns:
        Dict[str, Any]: Clasificación con los siguientes campos:
            - tipo_documento: str (código TD*)
            - tipo_hecho: List[str] (códigos TH*)
            - territorio: List[str] (nombres de departamentos)
            - periodo: str (código PER*)
            - actores: List[str] (códigos ACT*)
            - ruteo: str (código RU*)
            - highlights: List[str] (fragmentos textuales)
            - priority_score: float (0.0 a 1.0)
    
    Raises:
        ValueError: Si el modelo no retorna un JSON válido
        openai.APIError: Si hay problemas de comunicación con la API
        
    Ejemplo:
        >>> resultado = classify_document("""
        ...     Mi hermano Juan desapareció en 1998 en Urabá.
        ...     Unos hombres de las autodefensas se lo llevaron.
        ... """)
        >>> resultado["tipo_documento"]
        'TD2'  # Testimonio de familiar
        >>> resultado["tipo_hecho"]
        ['TH1']  # Desaparición forzada
        >>> resultado["actores"]
        ['ACT1']  # Paramilitares
    """
    # -------------------------------------------------------------------------
    # Paso 1: Preprocesamiento
    # Normaliza Unicode, elimina espacios múltiples, limpia headers/footers
    # -------------------------------------------------------------------------
    clean_text = preprocess_text(text)
    
    # -------------------------------------------------------------------------
    # Paso 2: Construcción del prompt
    # Inserta el documento limpio en el template con ejemplos few-shot
    # -------------------------------------------------------------------------
    user_prompt = build_user_prompt(clean_text)
    
    # -------------------------------------------------------------------------
    # Paso 3: Llamada al LLM
    # Envía system prompt (ontología + reglas) y user prompt (documento)
    # -------------------------------------------------------------------------
    raw_response = call_llm(SYSTEM_PROMPT, user_prompt)
    
    # -------------------------------------------------------------------------
    # Paso 4: Parsing de la respuesta
    # Extrae el JSON del texto de respuesta y lo convierte a dict
    # -------------------------------------------------------------------------
    raw_pred = parse_model_response(raw_response)
    
    # -------------------------------------------------------------------------
    # Paso 5: Validación y enriquecimiento
    # Corrige códigos inválidos, aplica reglas de negocio, calcula prioridad
    # -------------------------------------------------------------------------
    final_pred = validate_and_fix(raw_pred)
    
    return final_pred

---
## 13. Ejemplo de uso con testimonio sintético

Esta celda demuestra el uso del clasificador con un **testimonio de prueba** que incluye los elementos típicos de un relato de víctima.

### Elementos del testimonio de ejemplo:

| Elemento | Valor en el texto | Clasificación esperada |
|----------|-------------------|------------------------|
| Tipo | Primera persona, víctima directa | TD1 |
| Hecho 1 | "se llevaron a mi esposo" | TH1 (Desaparición) |
| Hecho 2 | "nos tocó salir" | TH3 (Desplazamiento) |
| Territorio | "San Carlos, Antioquia" | ["Antioquia"] |
| Período | "1997" | PER2 (1985-2000) |
| Actor | "guerrilla" | ACT2 |
| Ruteo | Desaparición → prioritario | RU1 |

### Resultado esperado:

```json
{
  "tipo_documento": "TD1",
  "tipo_hecho": ["TH1", "TH3"],
  "territorio": ["Antioquia"],
  "periodo": "PER2",
  "actores": ["ACT2"],
  "ruteo": "RU1",
  "highlights": ["..."],
  "priority_score": 0.7
}
```

El `priority_score` de 0.7 resulta de: TH1 (0.4) + RU1 (0.3) = 0.7

In [ ]:
# ==============================================================================
# SECCIÓN 13: EJEMPLO DE USO
# ==============================================================================
# Propósito:
#   Demostrar el uso del clasificador con un testimonio de prueba.
#   Este ejemplo es SINTÉTICO y no corresponde a un caso real.
#
# El testimonio de ejemplo contiene elementos típicos:
#   - Narrador en primera persona (víctima directa o familiar)
#   - Referencia temporal ("1997")
#   - Ubicación geográfica ("San Carlos, Antioquia")
#   - Actor armado ("guerrilla")
#   - Hechos victimizantes (desaparición, desplazamiento)
#
# Resultado esperado:
#   tipo_documento: TD1 (testimonio de víctima directa)
#   tipo_hecho: [TH1, TH3] (desaparición forzada, desplazamiento)
#   territorio: [Antioquia]
#   periodo: PER2 (1985-2000)
#   actores: [ACT2] (guerrilla)
#   ruteo: RU1 (equipo prioritario por TH1)
#   priority_score: 0.7 (TH1=0.4 + RU1=0.3)
# ==============================================================================

# Testimonio sintético de prueba
# NOTA: Este texto es ficticio y se usa solo para demostración.
# En producción se usarían documentos reales de la UBPD.

testimonio = """
Yo, María, cuento que en 1997, en el municipio de San Carlos, Antioquia, hombres armados
que se identificaron como de la guerrilla se llevaron a mi esposo. Desde ese día no volvimos
a saber de él. Después de eso comenzaron las amenazas y nos tocó salir de la vereda e irnos
para Medellín, dejando todo atrás.
"""

# Ejecutar clasificación
resultado = classify_document(testimonio)

# Mostrar resultado
# La salida incluirá todos los campos de clasificación + priority_score
resultado

---
## 14. Interpretación del resultado

El diccionario retornado contiene toda la información necesaria para:

### Uso inmediato:
- **Enrutamiento**: `ruteo` indica el equipo asignado.
- **Priorización**: `priority_score` permite ordenar casos.
- **Contexto rápido**: `highlights` muestra los fragmentos más relevantes.

### Análisis posterior:
- **Estadísticas territoriales**: Agregar por `territorio`.
- **Patrones temporales**: Analizar por `periodo`.
- **Mapeo de actores**: Cruzar `actores` con bases de datos externas.

### Integración con sistemas:

```python
# Ejemplo: Insertar en base de datos
INSERT INTO clasificaciones (
    documento_id,
    tipo_documento,
    hechos,
    territorio,
    periodo,
    actores,
    ruteo,
    priority_score
) VALUES (
    :doc_id,
    :resultado['tipo_documento'],
    :json.dumps(resultado['tipo_hecho']),
    :json.dumps(resultado['territorio']),
    :resultado['periodo'],
    :json.dumps(resultado['actores']),
    :resultado['ruteo'],
    :resultado['priority_score']
)
```

---
## 15. Próximos pasos y mejoras potenciales

Este prototipo es funcional pero hay múltiples oportunidades de mejora:

### Corto plazo:
- [ ] Agregar manejo de errores más robusto (retry logic, timeouts)
- [ ] Implementar logging estructurado para auditoría
- [ ] Validar ontología contra documentos reales de la UBPD
- [ ] Agregar más ejemplos few-shot (casos límite)

### Mediano plazo:
- [ ] Implementar batch processing para múltiples documentos
- [ ] Agregar métricas de calidad (accuracy, F1 por categoría)
- [ ] Crear dashboard de monitoreo
- [ ] Integrar con base de datos PostgreSQL

### Largo plazo:
- [ ] Fine-tuning de modelo con datos etiquetados de la UBPD
- [ ] Implementar explicabilidad (por qué se eligió cada etiqueta)
- [ ] Agregar clasificación multi-documento (casos relacionados)
- [ ] Integrar con sistemas de búsqueda semántica (embeddings + vector DB)

---
**Notebook preparado por:** Manuel Daza Ramirez  
Rol: AI Engineer (prototipo de clasificación de documentos testimoniales)  
Versión: 2025-02  

---

### Licencia y uso

Este notebook es un **prototipo de demostración** desarrollado para mostrar capacidades de clasificación automática de documentos testimoniales usando LLMs.

**Contacto:**
- LinkedIn: [linkedin.com/in/manueldazaramirez](https://linkedin.com/in/manueldazaramirez)
- Email: manuel.dazaramirez@gmail.com

---